# Autocorrección

A continuación vamos a resumir la primera semana del segundo curso de la especialización de NLP: https://www.coursera.org/specializations/natural-language-processing

El segundo curso se llama: Probabilistic Models
La primera semana: Autocorrect

## ¿Qué es autocorrección?

Es una aplicación que cambia palabras incorrectas en palabras correctas.

In [1]:
from IPython.display import Image
import pickle
import pandas as pd

In [2]:
PATH_PALABRAS = 'D:/01_NLP/NLP_aplicaciones/Probabilistic_models/Autocorrect/filename.pickle'

In [3]:
Image(url="https://raw.githubusercontent.com/Jazielinho/NLP_especialization/master/01_Classification_and_vector_spaces/01_Sentiment_Analysis_with_Logistic_Regression/01_resumen/01_machine_learning.PNG")

## ¿Cómo corregir automáticamente una palabra incorrecta?

1. Identificar la palabra incorrecta

2. Encontrar candidatos con distancia = 'n'

3. Filtrar candidatos

4. Calcular probabilidades de los candidatos

In [4]:
dict_palabras_cantidad = pickle.load(open(PATH_PALABRAS, 'rb'))
dict_palabras_cantidad = {str(k).lower(): v for k, v in dict_palabras_cantidad.items()}
palabras_validas = set(dict_palabras_cantidad.keys())
cantidad_total = sum(dict_palabras_cantidad.values())
dict_palabras_prob = {k: v / cantidad_total for k, v in dict_palabras_cantidad.items()}

In [5]:
LETRAS = 'abcdefghijklmnopqrstuvwxyz'

## Identificar palabra incorrecta

¿cómo saber si una palabra está escrita de manera incorrecta?
No se puede encontrar la palabra en un diccionario

In [6]:
def verifica_palabra(palabra):
    if palabra in palabras_validas:
        return True
    return False  

In [7]:
verifica_palabra('ejemplo')

True

In [8]:
verifica_palabra('ejemplho')

False

## Encontrar candidatos de distancia = 'n'

### Insertar

In [9]:
def insertar(palabra):
    ''' Insertamos una letra dentro de la palabra original '''
    lista_nuevas_palabras = []
    for i in range(len(palabra) + 1):
        for letra in LETRAS:
            nueva_palabra = palabra[:i] + letra + palabra[i:]
            lista_nuevas_palabras.append(nueva_palabra)
    return lista_nuevas_palabras

In [10]:
insertar('ejemplho')[:10]

['aejemplho',
 'bejemplho',
 'cejemplho',
 'dejemplho',
 'eejemplho',
 'fejemplho',
 'gejemplho',
 'hejemplho',
 'iejemplho',
 'jejemplho']

### Eliminar

In [11]:
def eliminar(palabra):
    ''' eliminamos una letra de la palabra original '''
    lista_nuevas_palabras = []
    for i in range(len(palabra)):
        nueva_palabra = palabra[:i] + palabra[(i+1):]
        lista_nuevas_palabras.append(nueva_palabra)
    return lista_nuevas_palabras

In [12]:
eliminar('ejemplho')

['jemplho',
 'eemplho',
 'ejmplho',
 'ejeplho',
 'ejemlho',
 'ejempho',
 'ejemplo',
 'ejemplh']

### Transponer

In [13]:
def transponer(palabra):
    ''' intercambiamos dos letras adyacentes'''
    lista_nuevas_palabras = []
    for i in range(len(palabra) - 1):
        nueva_palabra = palabra[:i] + palabra[(i+1)] + palabra[i] + palabra[(i+2):]
        lista_nuevas_palabras.append(nueva_palabra)
    return lista_nuevas_palabras

### Reemplazar

In [14]:
def reemplazar(palabra):
    ''' reemplazamos una 1 letra '''
    lista_nuevas_palabras = []
    for i in range(len(palabra)):
        for letra in LETRAS:
            nueva_palabra = palabra[:i] + letra + palabra[(i+1):]
            lista_nuevas_palabras.append(nueva_palabra)
    return lista_nuevas_palabras

In [15]:
reemplazar('ejemplho')[:10]

['ajemplho',
 'bjemplho',
 'cjemplho',
 'djemplho',
 'ejemplho',
 'fjemplho',
 'gjemplho',
 'hjemplho',
 'ijemplho',
 'jjemplho']

### Combinando y filtrando candidatos

In [16]:
def candidatos(palabra):
    ''' ediciones: insertar, eliminar, transponer, reemplazar '''
    lista_insertar = insertar(palabra)
    lista_eliminar = eliminar(palabra)
    lista_transponer = transponer(palabra)
    lista_reemplazar = reemplazar(palabra)
    lista_nuevas_palabras = lista_insertar + lista_eliminar + lista_transponer + lista_reemplazar
    ''' eliminamos duplicados '''
    lista_nuevas_palabras = set(lista_nuevas_palabras)
    ''' convertimos en lista '''
    lista_nuevas_palabras = list(lista_nuevas_palabras)
    ''' devolvemos las que están en el diccionario'''
    return [x for x in lista_nuevas_palabras if verifica_palabra(x)]

In [17]:
candidatos('ejemplho')[:10]

['ejemplo']

### Calculando probabilidades

In [18]:
def mejores_recomendaciones(lista_palabras, n=5):
    diccionario_prob = {k: dict_palabras_prob[k] for k in lista_palabras}
    dict_mejores_recomendaciones = pd.Series(diccionario_prob).sort_values(ascending=False).head(n).to_dict()
    return dict_mejores_recomendaciones   

In [19]:
mejores_recomendaciones(candidatos('ejemplho'))

{'ejemplo': 0.00033956211504374684}

## Poniendo todo junto

In [20]:
def corrige(palabra, n=5):
    palabra = str(palabra).lower()
    
    if verifica_palabra(palabra):
        return {palabra: dict_palabras_prob[palabra]}
    
    lista_candidatos = candidatos(palabra)
    
    if len(lista_candidatos) == 0:
        print('No tenemos candidatos para la palabra {}'.format(palabra))
        return None
    
    print('Hemos encontrado {} palabras válidas'.format(len(lista_candidatos)))
    
    dict_mejores_reco = mejores_recomendaciones(lista_candidatos, n)
    
    return dict_mejores_reco

In [21]:
corrige('ejemplho')

Hemos encontrado 1 palabras válidas


{'ejemplo': 0.00033956211504374684}

In [22]:
corrige('holah')

Hemos encontrado 6 palabras válidas


{'hola': 1.1405479994906077e-05,
 'holan': 7.865848272349018e-08,
 'holas': 5.243898848232679e-08,
 'olah': 1.9664620680872544e-08,
 'holaa': 6.554873560290849e-09}

In [23]:
corrige('123')

No tenemos candidatos para la palabra 123
